## Attempting Base Paper Implementation: MVI-MQFS

In [ ]:
!pip install qiskit qiskit_aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from PIL import Image
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import warnings

In [ ]:
def text_to_bits(text):
    """Convert text to a string of bits."""
    return ''.join(format(ord(char), '08b') for char in text)

def bits_to_text(bits):
    """Convert a string of bits back to text."""
    padding = 8 - (len(bits) % 8)
    if padding != 8:
        bits = "0" * padding + bits

    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    text = ""
    for char_bits in chars:
        if int(char_bits, 2) != 0:
            text += chr(int(char_bits, 2))
    return text

In [ ]:
def image_to_bitstream(image_path):
    """Load an image and convert it into a flat bitstream."""
    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image)
    return ''.join(format(val, '08b') for val in image_array.flatten())

In [ ]:
def bitstream_to_image(bitstream, image_path, output_path):
    """Convert a bitstream back into an image of the same dimensions as a template."""
    template_image = Image.open(image_path).convert('RGB')
    width, height = template_image.size

    num_values = width * height * 3
    expected_bits = num_values * 8

    if len(bitstream) > expected_bits:
        bitstream = bitstream[:expected_bits]
    elif len(bitstream) < expected_bits:
        bitstream += '0' * (expected_bits - len(bitstream))

    pixel_values = [int(bitstream[i:i+8], 2) for i in range(0, expected_bits, 8)]
    image_array = np.array(pixel_values, dtype=np.uint8).reshape((height, width, 3))
    stego_image = Image.fromarray(image_array)
    stego_image.save(output_path)

In [ ]:
def quantum_xor_operation(bit1, bit2):
    """
    Performs a symmetric and deterministic quantum XOR (CX) operation.
    This is used for both embedding (stego = host XOR secret)
    and extraction (secret = stego XOR host).
    """
    qc = QuantumCircuit(2, 1)

    if bit1 == '1':
        qc.x(0)
    if bit2 == '1':
        qc.x(1)

    qc.cx(1, 0)

    qc.measure(0, 0)

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1, memory=True).result()
    output_bit = result.get_memory()[0]

    return output_bit

In [ ]:
def encrypt_with_qiskit(image_path, secret_message, modulo_value, output_path):
    """
    Encrypts a message into an image using the MVI-MQFS conceptual model.
    """
    print("--- Starting Encryption ---")

    image_bits = list(image_to_bitstream(image_path))
    secret_bits = text_to_bits(secret_message)

    msg_len_bits = format(len(secret_bits), '032b')
    for i in range(32):
        image_bits[i] = msg_len_bits[i]

    print(f"Embedding {len(secret_bits)} bits of secret data...")

    for i in range(len(secret_bits)):
        position = 32 + (i * modulo_value)

        if position >= len(image_bits):
            raise ValueError("Message is too long or modulo value is too large for this image.")

        host_bit = image_bits[position]
        secret_bit = secret_bits[i]

        stego_bit = quantum_xor_operation(host_bit, secret_bit)

        image_bits[position] = stego_bit

        if (i+1) % 50 == 0:
            print(f"  ...embedded bit {i+1}/{len(secret_bits)}")

    bitstream_to_image("".join(image_bits), image_path, output_path)
    print(f"Encryption complete. Stego image saved to '{output_path}'")


In [ ]:
def decrypt_with_qiskit(stego_image_path, original_cover_path, modulo_value):
    """
    Decrypts a message from an image using the non-blind MVI-MQFS model.
    Requires the original cover image for comparison.
    """
    print("\n--- Starting Decryption ---")

    stego_bits = image_to_bitstream(stego_image_path)
    original_bits = image_to_bitstream(original_cover_path)

    msg_len_bits = stego_bits[:32]
    msg_len = int(msg_len_bits, 2)

    print(f"Expecting a message of {msg_len} bits...")

    extracted_secret_bits = ""
    for i in range(msg_len):
        position = 32 + (i * modulo_value)

        if position >= len(stego_bits):
            raise ValueError("Image is not large enough to contain the encoded message length.")

        stego_bit = stego_bits[position]
        host_bit = original_bits[position]

        secret_bit = quantum_xor_operation(stego_bit, host_bit)

        extracted_secret_bits += secret_bit

        if (i+1) % 50 == 0:
            print(f"  ...extracted bit {i+1}/{msg_len}")

    secret_message = bits_to_text(extracted_secret_bits)
    print(f"Decryption complete.")
    return secret_message

In [ ]:
COVER_IMAGE = "cover_image.png"
STEGO_IMAGE = "stego_image_qiskit.png"
SECRET_MESSAGE = "This is a secret test of the Qisk-it MVI-MQFS conceptual model!"

MODULO_VALUE = 313
encrypt_with_qiskit(COVER_IMAGE, SECRET_MESSAGE, MODULO_VALUE, STEGO_IMAGE)

decrypted_message = decrypt_with_qiskit(STEGO_IMAGE, COVER_IMAGE, MODULO_VALUE)

print("\n--- Verification ---")
print("Original Message: ", SECRET_MESSAGE)
print("Decrypted Message:", decrypted_message)

if SECRET_MESSAGE == decrypted_message:
    print("\nSuccess! The message was recovered correctly.")
else:
    print("\nFailure. The message was not recovered correctly.")

--- Starting Encryption ---
Embedding 504 bits of secret data...
  ...embedded bit 50/504
  ...embedded bit 100/504
  ...embedded bit 150/504
  ...embedded bit 200/504
  ...embedded bit 250/504
  ...embedded bit 300/504
  ...embedded bit 350/504
  ...embedded bit 400/504
  ...embedded bit 450/504
  ...embedded bit 500/504
Encryption complete. Stego image saved to 'stego_image_qiskit.png'

--- Starting Decryption ---
Expecting a message of 504 bits...
  ...extracted bit 50/504
  ...extracted bit 100/504
  ...extracted bit 150/504
  ...extracted bit 200/504
  ...extracted bit 250/504
  ...extracted bit 300/504
  ...extracted bit 350/504
  ...extracted bit 400/504
  ...extracted bit 450/504
  ...extracted bit 500/504
Decryption complete.

--- Verification ---
Original Message:  This is a secret test of the Qisk-it MVI-MQFS conceptual model!
Decrypted Message: This is a secret test of the Qisk-it MVI-MQFS conceptual model!

Success! The message was recovered correctly.


In [ ]:
from IPython.display import display

COVER_IMAGE = "/content/cover_image.png"
STEGO_IMAGE = "/content/stego_image_qiskit.png"

print("Cover Image:")
cover_image = Image.open(COVER_IMAGE)
display(cover_image)

print("\nStego Image:")
stego_image = Image.open(STEGO_IMAGE)
display(stego_image)

## Novelty Attempt

In [ ]:
!pip install numpy pillow qiskit qiskit-aer torch torchvision opencv-python --quiet

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import cv2

PROCESSING_SIZE = (384, 384)

In [ ]:
def text_to_bits(text):
    """Convert text to a string of bits."""
    return ''.join(format(ord(char), '08b') for char in text)

def bits_to_text(bits):
    """Convert a string of bits back to text."""
    padding = 8 - (len(bits) % 8)
    if padding != 8: bits = "0" * padding + bits
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    text = ""
    for char_bits in chars:
        if int(char_bits, 2) != 0: text += chr(int(char_bits, 2))
    return text

def quantum_xor_operation(bit1, bit2):
    """Performs a deterministic quantum XOR (CX) operation."""
    qc = QuantumCircuit(2, 1)
    if bit1 == '1': qc.x(0)
    if bit2 == '1': qc.x(1)
    qc.cx(1, 0)
    qc.measure(0, 0)
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1, memory=True).result()
    return result.get_memory()[0]

In [ ]:
def get_salience_map(image, threshold=0.5):
    """
    Generates a salience map from a pre-resized PIL image.
    The output mask will have the same dimensions as the input image.
    """
    try:
        model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small", trust_repo=True)
    except Exception as e:
        print(f"Failed to load AI model. Please check internet connection. Error: {e}")
        return None
    model.to('cpu')
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_batch = transform(image).unsqueeze(0)

    with torch.no_grad():
        prediction = model(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=image.size[::-1],
            mode="bicubic", align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()
    normalized_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
    binary_mask = (normalized_map < threshold).astype(np.uint8)

    print(f"Salience map generated with dimensions: {binary_mask.shape}")
    return binary_mask.flatten()

In [ ]:
def get_salience_map(image_path, threshold=0.5, alpha=0.4):
    """
    Uses a pre-trained model to generate a salience map and saves a visual overlay.
    This version GUARANTEES the output mask matches the input image dimensions
    and also saves a highlighted visualization image.
    """
    print("--- 1. Generating Classical Salience Map ---")
    try:
        model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small", trust_repo=True)
    except Exception as e:
        print(f"Failed to load AI model. Please check internet connection. Error: {e}")
        return None
    model.to('cpu')
    model.eval()

    input_image_path = "./cover_image.png"
    input_image = Image.open(input_image_path).convert('RGB')
    original_size_wh = input_image.size
    original_size_hw = (input_image.height, input_image.width)

    processed_image = input_image.resize(PROCESSING_SIZE, Image.Resampling.LANCZOS)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_batch = transform(processed_image).unsqueeze(0)

    with torch.no_grad():
        prediction = model(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=original_size_hw,
            mode="bicubic", align_corners=False,
        ).squeeze()


    depth_map = prediction.cpu().numpy()
    normalized_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

    saliency_map_resized = normalized_map

    if saliency_map_resized.shape != original_size_hw:
        print(f"Warning: Correcting salience map dimension mismatch. Finalizing to {original_size_hw}")
        saliency_map_resized = cv2.resize(saliency_map_resized, original_size_wh, interpolation=cv2.INTER_NEAREST)


    binary_mask = (saliency_map_resized < threshold).astype(np.uint8)
    print(f"Salience map generated with guaranteed dimensions: {binary_mask.shape}")

    overlay_array = np.zeros((original_size_hw[0], original_size_hw[1], 3), dtype=np.uint8)
    overlay_array[binary_mask == 1] = [255, 0, 0]
    overlay_image = Image.fromarray(overlay_array, 'RGB')

    highlighted_image = Image.blend(input_image, overlay_image, alpha=alpha)
    highlighted_image.save("salience_overlay_visualization.png")
    print("Salience overlay visualization saved as 'salience_overlay_visualization.png'")


    return binary_mask.flatten()

In [ ]:
def encrypt(image_path, secret_message, output_path):

    original_image = Image.open(image_path).convert('RGB')
    processed_image = original_image.resize(PROCESSING_SIZE, Image.Resampling.LANCZOS)

    image_array = np.array(processed_image)
    image_bits = list(''.join(format(val, '08b') for val in image_array.flatten()))

    safe_pixel_mask = get_salience_map(processed_image)
    if safe_pixel_mask is None: return

    safe_pixel_indices = np.where(safe_pixel_mask == 1)[0]

    available_embedding_indices = []
    for pixel_idx in safe_pixel_indices:
        available_embedding_indices.extend([pixel_idx * 24 + 7, pixel_idx * 24 + 15, pixel_idx * 24 + 23])

    secret_bits = text_to_bits(secret_message)
    msg_len_bits = format(len(secret_bits), '032b')
    full_secret_bits = msg_len_bits + secret_bits

    if len(full_secret_bits) > len(available_embedding_indices):
        raise ValueError(f"Message too long for available safe LSBs. Need {len(full_secret_bits)}, have {len(available_embedding_indices)}.")

    print(f"\nEmbedding {len(full_secret_bits)} bits of data into safe LSBs...")

    for i in range(len(full_secret_bits)):
        position = available_embedding_indices[i]
        host_bit, secret_bit = image_bits[position], full_secret_bits[i]
        stego_bit = quantum_xor_operation(host_bit, secret_bit)
        image_bits[position] = stego_bit

    stego_array = np.array([int("".join(image_bits[i:i+8]), 2) for i in range(0, len(image_bits), 8)], dtype=np.uint8).reshape(image_array.shape)
    stego_image = Image.fromarray(stego_array)
    stego_image.save(output_path)
    print(f"Encryption complete. Stego image saved to '{output_path}'")

In [ ]:
def decrypt(stego_image_path, original_cover_path):
    original_image = Image.open(original_cover_path).convert('RGB')
    processed_original = original_image.resize(PROCESSING_SIZE, Image.Resampling.LANCZOS)

    stego_image = Image.open(stego_image_path).convert('RGB')
    if stego_image.size != PROCESSING_SIZE:
        print(f"Warning: Stego image size is {stego_image.size}, resizing to standard {PROCESSING_SIZE} for decryption.")
        stego_image = stego_image.resize(PROCESSING_SIZE, Image.Resampling.LANCZOS)

    stego_bits = ''.join(format(val, '08b') for val in np.array(stego_image).flatten())
    original_bits = ''.join(format(val, '08b') for val in np.array(processed_original).flatten())

    safe_pixel_mask = get_salience_map(processed_original)
    if safe_pixel_mask is None: return

    safe_pixel_indices = np.where(safe_pixel_mask == 1)[0]
    available_embedding_indices = []
    for pixel_idx in safe_pixel_indices:
        available_embedding_indices.extend([pixel_idx * 24 + 7, pixel_idx * 24 + 15, pixel_idx * 24 + 23])

    msg_len_bits = ""
    for i in range(32):
        position = available_embedding_indices[i]
        stego_bit, host_bit = stego_bits[position], original_bits[position]
        len_bit = quantum_xor_operation(stego_bit, host_bit)
        msg_len_bits += len_bit
    msg_len = int(msg_len_bits, 2)
    print(f"Expecting a message of {msg_len} bits...")

    extracted_secret_bits = ""
    for i in range(msg_len):
        position = available_embedding_indices[32 + i]
        stego_bit, host_bit = stego_bits[position], original_bits[position]
        secret_bit = quantum_xor_operation(stego_bit, host_bit)
        extracted_secret_bits += secret_bit

    secret_message = bits_to_text(extracted_secret_bits)
    print(f"Decryption complete.")
    return secret_message

In [ ]:
COVER_IMAGE = "cover_image.png"
STEGO_IMAGE = "stego_image_hybrid.png"
SECRET_MESSAGE = "This is the final test for the hybrid classical-quantum steganography model!"

encrypt(COVER_IMAGE, SECRET_MESSAGE, STEGO_IMAGE)
decrypted_message = decrypt(STEGO_IMAGE, COVER_IMAGE)

print("Original Message:  ", SECRET_MESSAGE)
print("Decrypted Message: ", decrypted_message)

if SECRET_MESSAGE == decrypted_message:
    print("\nThe message was recovered correctly.")
else:
    print("\nThe message was not recovered correctly.")

--- 1. Generating Classical Salience Map ---


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Loading weights:  None
Salience map generated with guaranteed dimensions: (2000, 3008)


/tmp/ipython-input-2869777183.py:53: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  overlay_image = Image.fromarray(overlay_array, 'RGB')


Salience overlay visualization saved as 'salience_overlay_visualization.png'

Embedding 640 bits of data into safe LSBs...
Encryption complete. Stego image saved to 'stego_image_hybrid.png'
--- 1. Generating Classical Salience Map ---


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Loading weights:  None
Salience map generated with guaranteed dimensions: (2000, 3008)
Salience overlay visualization saved as 'salience_overlay_visualization.png'
Expecting a message of 608 bits...
Decryption complete.
Original Message:   This is the final test for the hybrid classical-quantum steganography model!
Decrypted Message:  This is the final test for the hybrid classical-quantum steganography model!

The message was recovered correctly.


In [ ]:
from IPython.display import display

COVER_IMAGE = "/content/cover_image.png"
SALIENCE_IMAGE = "/content/salience_overlay_visualization.png"
STEGO_IMAGE = "/content/stego_image_hybrid.png"

print("Cover Image:")
cover_image = Image.open(COVER_IMAGE)
display(cover_image)

print("\nSalience Map:")
salience_image = Image.open(SALIENCE_IMAGE)
display(salience_image)

print("Stego Image:")
stego_image = Image.open(STEGO_IMAGE)
display(stego_image)